# 1.0 Install and import the required libraries

In [1]:
# !pip install neo4j-runway
# !pip install langchain-groq

In [2]:
import numpy as np
import pandas as pd
from neo4j_runway import Discovery, GraphDataModeler, IngestionGenerator, PyIngest
from neo4j_runway.llm.openai import OpenAIDiscoveryLLM 

# 2.0 Read the CSV file and create a smaller subset 
- the dataset is a pretty big so ingesting it into neo4j takes too long with my current hardware
- therefore I created a smaller subset 

In [3]:
# Read the CSV file
file_path = 'C:/Users/hezro/Desktop/data.csv'
df = pd.read_csv(file_path)

# Remove NULL values and replace them with 'Unknown', because you can't ingest data into neo4j if there's NULL values
df['merchant'] = df['merchant'].fillna('Unknown')

# Change the column 'txn_date' to date and time datatype
df['txn_date'] = pd.to_datetime(df['txn_date'], format='%d/%m/%Y %H:%M')

# Split the column 'txn_date' to 'txn_date' and 'txn_time'
df['txn_date'], df['txn_time'] = df['txn_date'].dt.date, df['txn_date'].dt.time

# Sort client ID in the dataframe to ascending order
df = df.sort_values(by='clnt_id', ascending=True)

# Get the first 500 rows only
df_500 = df.head(500)

# Select the first 10 rows where 'clnt_id' is 880 so that we can have some negative values in column 'amt', to test for money spent
client_id_880 = df[df['clnt_id'] == 880].head(10)

# Append those rows back to the DataFrame
df_500 = pd.concat([df_500, client_id_880], ignore_index=True)

# df_500.tail(5)

# 3.0 Connect to neo4j (Online / Local) 

In [4]:
from neo4j import GraphDatabase

# Using online database
neo4j_uri = "neo4j+s://e975d7ed.databases.neo4j.io"
neo4j_user = "neo4j"
neo4j_password = "hRlFcTRimOntJrGYOAWYpwJHZAgizQhAx9q_0A9YhNA"
driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_password))

# # Using local neo4j database, note that you need to install neo4j and create a new database
# neo4j_uri = "neo4j://localhost:7687"
# neo4j_user = "neo4j"
# neo4j_password = "PASSWORD"
# driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_password))

# 4.0 Cypher query to ingest data

## 4.1 Cypher query

In [5]:
def ingest_data(tx, row):
    tx.run('''
            CREATE (transaction:Transaction 
            {
                transactionID: $txn_id,
                transactionDate: date($txn_date),
                description: $desc,
                amount: $amt,
                category: $cat 
            })
            
            MERGE (client:Client {clientID: $clnt_id})
            MERGE (bank:Bank {bankID: $bank_id})
            MERGE (account:Account {accountID: $acc_id})
            MERGE (merchant:Merchant {name: $merchant})

            // Create a relationship between the client and the transaction
            CREATE (client)-[:MADE_TRANSACTION {amount: $amt}]->(transaction)

            // Create a relationship between the merchant and the transaction
            CREATE (merchant)-[:PROCESSED_TRANSACTION]->(transaction)

            CREATE (transaction)-[:BELONGS_TO_ACCOUNT]->(account)
            CREATE (account)-[:HAS_ACCOUNT_AT]->(bank)
    ''',
    txn_id=row['txn_id'],
    txn_date=row['txn_date'],
    desc=row['desc'],
    amt=row['amt'],
    cat=row['cat'],
    clnt_id=row['clnt_id'],
    bank_id=row['bank_id'],
    acc_id=row['acc_id'],
    merchant=row['merchant']
    )


## 4.2 Start ingesting data

In [6]:
# # Ingest the data into Neo4j
# with driver.session() as session:
#     for _, row in df_500.iterrows():
#         session.write_transaction(ingest_data, row.to_dict())

# 5.0 Enter Groq API and choose the desired model

In [7]:
from langchain_groq import ChatGroq

# Enter your groq API here
groq_api = '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'

# Choose your model
llm = ChatGroq(temperature=0, model="llama-3.1-70b-versatile", api_key=groq_api)

# 6.0 Prompt Engineer Part 1
- This section define a function that prompts the LLM to generate a simple cypher query based on the user's question
- Includes the Node Lables and Relationship Types to ensure correct variable names
- Includes few shot learning to provide some examples of simple cypher queries

In [8]:
gen_query_instructions = """
    Based on the following client ID, question and context, generate a simple cypher query 

    Client ID: {cID}
    
    Question: {question}

    Context: {context}

    Output only the cypher query, nothing else. 

    Remember when calculating amount spent, only include the amount smaller than 0.
    Remember when calculating amount received, only include the amount bigger than 0.
    
    For example if Client ID is 1: 
    - How much did I receive from Amazon?
    - MATCH (client:Client {{clientID: 1}})-[:MADE_TRANSACTION]->(transaction:Transaction)<-[:PROCESSED_TRANSACTION]-(merchant:Merchant)
      WHERE toLower(merchant.name) = toLower('Amazon') 
      AND transaction.amount > 0 
      RETURN SUM(transaction.amount) AS total_received

        
    - How much did I receive in June 2023? 
    - MATCH (client:Client {{clientID: 1}})-[:MADE_TRANSACTION]->(transaction:Transaction)
      WHERE transaction.amount > 0  // Include only amounts larger than 0
      AND transaction.transactionDate >= date('2023-06-01') 
      AND transaction.transactionDate < date('2023-07-01') 
      RETURN SUM(transaction.amount) AS total_received_june
"""

In [9]:
data_dictionary = """
    Node Labels: 
    - Transaction
    - Client
    - Bank
    - Account
    - Merchant
    
    Relationship Types: 
    - MADE_TRANSACTION
    - PROCESSED_TRANSACTION
    - BELONGS_TO_ACCOUNT
    - HAS_ACCOUNT_AT
"""

## 6.1 Generate Simple Cypher Query
- here we are using the LLM to generate a simple cypher query based on prompt engineering just now

In [10]:
def generate_cypher(question, input_id):
    query_prompt = gen_query_instructions.format(cID=input_id, context=data_dictionary, question=question)
    query = llm.invoke(query_prompt).content
    return query

# 7.0 Prompt Engineer Part 2
- This section defines the function to input the simple cypher query generated earlier, into Neo4j
- Then the result from the query is used as the context for the LLM to generate an final output

In [11]:
prompt_instructions = """
    Based on the following question and context, answer the question  

    Question: {question}

    Context: {context}

    Responses should be in one sentence. Be succinct. 
"""

In [12]:
# Function that passes the generated query earlier, to get the graph context from the graph database Neo4j
def get_graph_context(cypher_query):
    with driver.session() as session:
        result = session.run(cypher_query) 
 
        graph_context = []
        for record in result:
            graph_context.append(record) 

        return graph_context


# 8.0 Function to get the final output
- Function first passes the question and input_id (client's ID)
- Then uses the generate cypher to get the LLM generated cypher
- Then uses get_graph_context to get the query output in neo4j and use it as context
- Then store the prompt from 'prompt engineering part 2' into the final prompt

In [13]:
def get_response(question, input_id):
    query = generate_cypher(question, input_id)
    context = get_graph_context(query)
    final_prompt = prompt_instructions.format(context=context, question=question)
    response = llm.invoke(final_prompt).content
    return response

In [14]:
# Prompt for user ID
input_id = int(input("Please enter your Client's ID: "))

# Prompt for client ID again if ID is invalid
while input_id not in df['clnt_id'].values:
    print("Invalid ID. Please try again.")
    input_id = int(input("Please enter your ID: "))

Please enter your Client's ID:  2


In [17]:
question = "how much did I receive in September 2023"
output = get_response(question, input_id)
print(output)

You received $1119.40 in September 2023.
